In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime, date
plt.style.use('ggplot')

In [3]:
# Load Customer Demographics data from excel file

trans = pd.read_excel('Raw_data.xlsx', sheet_name='Transactions')
trans.head(5)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [4]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

### Dataset overview

In [5]:
print("Total records in the dataset: {}".format(trans.shape[0]))
print("Total columns in the dataset: {}".format(trans.shape[1]))

Total records in the dataset: 20000
Total columns in the dataset: 13


In [6]:
# select numeric columns
df_numeric = trans.select_dtypes(include=[np.number])
numberic_cols = df_numeric.columns.values
print("The numeric columns are: {}".format(numberic_cols))

# select non-numeric columns
df_non_numeric = trans.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print("The non-numeric columns are: {}".format(non_numeric_cols))


The numeric columns are: ['transaction_id' 'product_id' 'customer_id' 'online_order' 'list_price'
 'standard_cost' 'product_first_sold_date']
The non-numeric columns are: ['transaction_date' 'order_status' 'brand' 'product_line' 'product_class'
 'product_size']


In [12]:
trans['product_first_sold_date'].nunique()


100

### 1. Missing values check

In [13]:
# Total number of missing values

trans.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [14]:
# Percentage of missing values

trans.isnull().mean()*100

transaction_id             0.000
product_id                 0.000
customer_id                0.000
transaction_date           0.000
online_order               1.800
order_status               0.000
brand                      0.985
product_line               0.985
product_class              0.985
product_size               0.985
list_price                 0.000
standard_cost              0.985
product_first_sold_date    0.985
dtype: float64

We can observe that columns online_order, brand, product_line, product_class , product_size, standard_cost, product_first_sold_date have missing values

#### 1.1 Online order

In [16]:
trans['online_order'].value_counts()

online_order
1.0    9829
0.0    9811
Name: count, dtype: int64

In [18]:
most_freq_online_mode = trans['online_order'].mode()
most_freq_online_mode

0    1.0
Name: online_order, dtype: float64

### 3. Inconsistency Check in Data

We will check whether there is inconsistent data / typo error data is present in the categorical columns.
The columns to be checked are 'gender', 'wealth_segment', 'deceased_indicator', 'owns_car'

#### 3.1 Gender

In [32]:
cust_demo['gender'].value_counts()

gender
Female    2037
Male      1872
F            1
Femal        1
M            1
Name: count, dtype: int64

In [33]:
def replace_inconsistency_gender(gender_type):
    if gender_type == "F":
        return "Female"
    elif gender_type == "M":
        return "Male"
    elif gender_type == "Femal":
        return "Female"
    else:
        return gender_type

cust_demo['gender'] = cust_demo['gender'].apply(replace_inconsistency_gender)

In [34]:
cust_demo['gender'].value_counts()

gender
Female    2039
Male      1873
Name: count, dtype: int64

#### 3.2 Wealth segment

In [36]:
cust_demo['wealth_segment'].value_counts()

wealth_segment
Mass Customer        1954
High Net Worth        996
Affluent Customer     962
Name: count, dtype: int64

#### 3.3 Decreased indicator

In [37]:
cust_demo['deceased_indicator'].value_counts()

deceased_indicator
N    3910
Y       2
Name: count, dtype: int64

#### 3.4 Owns Car

In [38]:
cust_demo['owns_car'].value_counts()

owns_car
Yes    1974
No     1938
Name: count, dtype: int64

### 4. Duplication checks

In [41]:
# Count the number of unique values in the 'customer_id' column
cust_demo['customer_id'].nunique()

3912

In [44]:
cust_drop_id = cust_demo.drop('customer_id', axis=1).drop_duplicates()

print("Total unique records after drop customer_id(pk) and duplicates: {}".format(cust_drop_id.shape[0]))

Total unique records after drop customer_id(pk) and duplicates: 3912


Since both the numbers are same. There are no duplicate records in the dataset.

### 5. Export cleaned customer Demogerphic dataset to CSV

In [45]:
cust_demo.to_csv('CustomerDemographic_Cleaned.csv', index=False)